In [22]:
import pandas as pd 
import json
import re

##### Load Data

In [23]:
#path
data_080415 = "/Users/cheimamezdour/Projects/DQN-frameworQ/data/A4W_CONTROL/FCA_E_A4-W_09+0250/2008-04-15_infosCA37000035.csv"

In [24]:
data = pd.read_csv(data_080415)

In [25]:
data.head()

,2008-04-15 00:00:20,419;JOUR_669.NUMERO_JOUR;1;JOUR_669.TYPE;1;CA_37000035.MODE_REG_ACTIF;2;CA_37000035.MODE_CA_ACTIF;4;CA_37000035.ETAT_MODE_DEGRADE;0;CA_37000035.ETAT_MODE_OPERATEUR;1;CA_37000035.TYPE_CA;0;CA_37000035.STRATEGIE_EN_COURS.TYPE;3;CA_37000035.STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE;120;CA_37000035.STRATEGIE_EN_COURS.CYCLE.DUREE_VERT_1;120;CA_37000035.STRATEGIE_EN_COURS.CYCLE.DUREE_VERT_2;120;CA_37000035.STRATEGIE_EN_COURS.CYCLE.DEBIT_1;-1;CA_37000035.STRATEGIE_EN_COURS.CYCLE.DEBIT_2;-1;CA_37000035.STRATEGIE_EN_COURS.CYCLE.APPLICATION_AU_PLUS_TOT;0;CA_37000035.STRATEGIE_EN_COURS.CYCLE.DUREE_VALIDITE_LCR;180;CA_37000035.STRATEGIE_EN_COURS.HORO_FIN;1230764400;CODE_INVALIDE.ID;CODE_INVALIDE;CA_37000035.LISTE_SCENARII_ACTIVES;;CA_37000035.ETAT_REMONTEE_FA;0;CA_37000035.TO_ACCES;0;CA_37000035.ETAT_VEILLE;1;CA_37000035.ETAT_INHIBE;0;CA_37000035.ETAT_DISP_FONCT;0;CA_37000035.STRATEGIE_EN_COURS.CONSIGNE_EN_COURS;14;CA_37000035.STRATEGIE_EN_COURS.CONSIGNE_DEFAUT;14;CA_37000035.STRATEGIE_EN_COURS.CONSIGNE_OPE;-1;CA_37000035.STRATEGIE_EN_COURS.CONSIGNE_ANTISAT;0;ST_31003478.Q;900;ST_31003478.Q_LISSE;972.0;ST_31003478.T;0.0;ST_31003478.T_LISSE;0.0;ST_31003478.ETAT_CONGESTION;0;ST_31003477.Q;840;ST_31003477.Q_LISSE;876.0;ST_31003477.T;1.0;ST_31003477.T_LISSE;1.0;ST_31003477.ETAT_CONGESTION;0
0,2008-04-15 00:01:00,162;JOUR_669.NUMERO_JOUR;1;JOUR_669.TYPE;1;CA_...
1,2008-04-15 00:01:40,160;JOUR_669.NUMERO_JOUR;1;JOUR_669.TYPE;1;CA_...
2,2008-04-15 00:02:20,165;JOUR_669.NUMERO_JOUR;1;JOUR_669.TYPE;1;CA_...
3,2008-04-15 00:03:00,162;JOUR_669.NUMERO_JOUR;1;JOUR_669.TYPE;1;CA_...
4,2008-04-15 00:03:40,158;JOUR_669.NUMERO_JOUR;1;JOUR_669.TYPE;1;CA_...


##### Structuring data into a : 
- State: Include relevant columns that describe the current state of the system.
- Action: Identify any columns that describe actions taken.
- Reward: Specify columns that indicate the reward.
- Next State: Derive the next state from the state columns shifted by one row.

###### 1 - Read the File: Read the content of the CSV file and split it into lines. 
###### 2 - Process Each Line: Split each line into timestamp and key-value pairs.
###### 3 - Create DataFrame: Convert the processed data into a pandas DataFrame.
###### 4 - Rename Columns: Assign readable names to the columns.
###### 5 - Define State, Action, Reward, Next State: Extract and transform columns into the desired format.

In [26]:
# Read the file content
file_path = data_080415
with open(file_path, 'r') as file:
    content = file.read()

# Split the content by newlines to get individual lines
lines = content.split('\n')

# Process each line
data_processed = []
for line in lines:
    if line.strip():  # Skip any empty lines
        parts = line.split(';')
        timestamp = parts[0]
        data_parts = parts[1:]
        data_processed.append([timestamp] + data_parts)

# Convert into a DataFrame
data_df = pd.DataFrame(data_processed)

#Define all the columns in the csv 
data_df.columns = [
    "timestamp", "JOUR.NUMERO_JOUR", "JOUR.NUMERO_JOUR.value", "JOUR.TYPE", "JOUR.TYPE.value", 
    "MODE_REG_ACTIF", "MODE_REG_ACTIF.value", "MODE_CA_ACTIF", "MODE_CA_ACTIF.value", 
    "ETAT_MODE_DEGRADE", "ETAT_MODE_DEGRADE.value", "ETAT_MODE_OPERATEUR", "ETAT_MODE_OPERATEUR.value", 
    "TYPE_CA", "TYPE_CA.value", "STRATEGIE_EN_COURS.TYPE", "STRATEGIE_EN_COURS.TYPE.value", 
    "STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE", "STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value", 
    "STRATEGIE_EN_COURS.CYCLE.DUREE_VERT_1", "STRATEGIE_EN_COURS.CYCLE.DUREE_VERT_1.value", 
    "STRATEGIE_EN_COURS.CYCLE.DUREE_VERT_2", "STRATEGIE_EN_COURS.CYCLE.DUREE_VERT_2.value", 
    "STRATEGIE_EN_COURS.CYCLE.DEBIT_1", "STRATEGIE_EN_COURS.CYCLE.DEBIT_1.value", 
    "STRATEGIE_EN_COURS.CYCLE.DEBIT_2", "STRATEGIE_EN_COURS.CYCLE.DEBIT_2.value", 
    "STRATEGIE_EN_COURS.CYCLE.APPLICATION_AU_PLUS_TOT", "STRATEGIE_EN_COURS.CYCLE.APPLICATION_AU_PLUS_TOT.value", 
    "STRATEGIE_EN_COURS.CYCLE.DUREE_VALIDITE_LCR", "STRATEGIE_EN_COURS.CYCLE.DUREE_VALIDITE_LCR.value", 
    "STRATEGIE_EN_COURS.HORO_FIN", "STRATEGIE_EN_COURS.HORO_FIN.value", 
    "CODE_INVALIDE.ID", "CODE_INVALIDE.ID.value", "LISTE_SCENARII_ACTIVES", "LISTE_SCENARII_ACTIVES.value", 
    "ETAT_REMONTEE_FA", "ETAT_REMONTEE_FA.value", "TO_ACCES", "TO_ACCES.value", 
    "ETAT_VEILLE", "ETAT_VEILLE.value", "ETAT_INHIBE", "ETAT_INHIBE.value", 
    "ETAT_DISP_FONCT", "ETAT_DISP_FONCT.value", "STRATEGIE_EN_COURS.CONSIGNE_EN_COURS", "STRATEGIE_EN_COURS.CONSIGNE_EN_COURS.value", 
    "STRATEGIE_EN_COURS.CONSIGNE_DEFAUT", "STRATEGIE_EN_COURS.CONSIGNE_DEFAUT.value", 
    "STRATEGIE_EN_COURS.CONSIGNE_OPE", "STRATEGIE_EN_COURS.CONSIGNE_OPE.value", 
    "STRATEGIE_EN_COURS.CONSIGNE_ANTISAT", "STRATEGIE_EN_COURS.CONSIGNE_ANTISAT.value", 
    "ST_31003478.Q", "ST_31003478.Q.value", "ST_31003478.Q_LISSE", "ST_31003478.Q_LISSE.value", 
    "ST_31003478.T", "ST_31003478.T.value", "ST_31003478.T_LISSE", "ST_31003478.T_LISSE.value", 
    "ST_31003478.ETAT_CONGESTION", "ST_31003478.ETAT_CONGESTION.value", "ST_31003477.Q", "ST_31003477.Q.value", 
    "ST_31003477.Q_LISSE", "ST_31003477.Q_LISSE.value", "ST_31003477.T", "ST_31003477.T.value", 
    "ST_31003477.T_LISSE", "ST_31003477.T_LISSE.value", "ST_31003477.ETAT_CONGESTION", "ST_31003477.ETAT_CONGESTION.value"
]

#Define the important columns for the learning
important_columns=[
    "STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value",  # Colonne T
    "STRATEGIE_EN_COURS.CYCLE.DUREE_VERT_1.value",  # Colonne V
    "STRATEGIE_EN_COURS.CONSIGNE_EN_COURS.value",  # Colonne AX
    'ST_31003477.T',  # Colonne BJ --Taux d'occupation
    'ST_31003477.T_LISSE',  # Colonne BL --Taux d'occupation
    'ST_31003477.Q',  # Colonne BT  -- Debit
    'ST_31003477.Q_LISSE'  # Colonne BV] --Debit
]

# Define columns for state, action, reward, and next state
state_cols = [
   
    "STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value", 
    "STRATEGIE_EN_COURS.CYCLE.DUREE_VERT_1.value",
    "ST_31003477.Q.value", 
    "ST_31003477.Q_LISSE.value",
    "ST_31003477.T.value",
    "ST_31003477.T_LISSE.value" 
]
action_cols = ["STRATEGIE_EN_COURS.CONSIGNE_EN_COURS.value"]
reward_col = "ST_31003477.ETAT_CONGESTION.value"

# Create the structured DataFrame
structured_df = pd.DataFrame()
structured_df['timestamp'] = data_df['timestamp']
structured_df['state'] = data_df[state_cols].apply(lambda row: row.to_dict(), axis=1)
structured_df['action'] = data_df[action_cols].apply(lambda row: row.to_dict(), axis=1)
structured_df['reward'] = data_df[reward_col]
structured_df['next_state'] = data_df[state_cols].shift(-1).apply(lambda row: row.to_dict(), axis=1)

# Drop the last row with NaN values in next_state
structured_df.dropna(inplace=True)

In [27]:
structured_df.head()

,timestamp,state,action,reward,next_state
0,"2008-04-15 00:00:20,419",{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...,{'STRATEGIE_EN_COURS.CONSIGNE_EN_COURS.value':...,0,{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...
1,"2008-04-15 00:01:00,162",{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...,{'STRATEGIE_EN_COURS.CONSIGNE_EN_COURS.value':...,0,{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...
2,"2008-04-15 00:01:40,160",{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...,{'STRATEGIE_EN_COURS.CONSIGNE_EN_COURS.value':...,0,{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...
3,"2008-04-15 00:02:20,165",{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...,{'STRATEGIE_EN_COURS.CONSIGNE_EN_COURS.value':...,0,{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...
4,"2008-04-15 00:03:00,162",{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...,{'STRATEGIE_EN_COURS.CONSIGNE_EN_COURS.value':...,0,{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...


## Save as CSV

In [28]:
structured_df.to_csv("./preprocessed_data/structured_data_080415.csv")

In [29]:
# Load the uploaded CSV file
file_path = './preprocessed_data/structured_data_080415.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataframe to understand its structure
df.head()

,Unnamed: 0,timestamp,state,action,reward,next_state
0,0,"2008-04-15 00:00:20,419",{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...,{'STRATEGIE_EN_COURS.CONSIGNE_EN_COURS.value':...,0,{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...
1,1,"2008-04-15 00:01:00,162",{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...,{'STRATEGIE_EN_COURS.CONSIGNE_EN_COURS.value':...,0,{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...
2,2,"2008-04-15 00:01:40,160",{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...,{'STRATEGIE_EN_COURS.CONSIGNE_EN_COURS.value':...,0,{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...
3,3,"2008-04-15 00:02:20,165",{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...,{'STRATEGIE_EN_COURS.CONSIGNE_EN_COURS.value':...,0,{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...
4,4,"2008-04-15 00:03:00,162",{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...,{'STRATEGIE_EN_COURS.CONSIGNE_EN_COURS.value':...,0,{'STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value':...


###  Removing the Key,value from the rows and keep just the numerical values

In [30]:
# Function to clean JSON strings and expand them into separate columns
def clean_and_expand_column(df, column_name):
    def clean_json_string(json_str):
        # Replace single quotes with double quotes
        json_str = json_str.replace("'", '"')
        # Remove any trailing commas before the closing brace
        json_str = re.sub(r',\s*}', '}', json_str)
        json_str = re.sub(r',\s*]', ']', json_str)
        return json_str

    def is_valid_json(json_str):
        try:
            json.loads(json_str)
            return True
        except ValueError:
            return False

    cleaned_column = df[column_name].apply(clean_json_string)
    valid_json_column = cleaned_column[cleaned_column.apply(is_valid_json)]
    expanded_df = valid_json_column.apply(json.loads).apply(pd.Series)
    expanded_df.columns = [f"{column_name}.{col}" for col in expanded_df.columns]
    return expanded_df

# Remove the first column if it is unnamed
if df.columns[0] == 'Unnamed: 0':
    df = df.drop(df.columns[0], axis=1)

# Clean and expand the 'state', 'action', and 'next_state' columns
state_expanded = clean_and_expand_column(df, 'state')
action_expanded = clean_and_expand_column(df, 'action')
next_state_expanded = clean_and_expand_column(df, 'next_state')

# Concatenate the expanded columns with the original dataframe
df_expanded = pd.concat([df.drop(['state', 'action', 'next_state'], axis=1), state_expanded, action_expanded, next_state_expanded], axis=1)

# Save the expanded dataframe to a new CSV file
output_file_path = './result_data/expanded_structured_data.csv'
df_expanded.to_csv(output_file_path, index=False)

# Output the file path for download
print("The expanded dataframe has been saved to:", output_file_path)


The expanded dataframe has been saved to: ./result_data/expanded_structured_data.csv


In [31]:
df_expanded.head(10)

,timestamp,reward,state.STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value,state.STRATEGIE_EN_COURS.CYCLE.DUREE_VERT_1.value,state.ST_31003477.Q.value,state.ST_31003477.Q_LISSE.value,state.ST_31003477.T.value,state.ST_31003477.T_LISSE.value,action.STRATEGIE_EN_COURS.CONSIGNE_EN_COURS.value,next_state.STRATEGIE_EN_COURS.CYCLE.DUREE_CYCLE.value,next_state.STRATEGIE_EN_COURS.CYCLE.DUREE_VERT_1.value,next_state.ST_31003477.Q.value,next_state.ST_31003477.Q_LISSE.value,next_state.ST_31003477.T.value,next_state.ST_31003477.T_LISSE.value
0,"2008-04-15 00:00:20,419",0,120,120,840,876.0,1.0,1.0,14,120,120,600,816.0,1.0,1.0
1,"2008-04-15 00:01:00,162",0,120,120,600,816.0,1.0,1.0,14,120,120,1200,840.0,3.0,3.0
2,"2008-04-15 00:01:40,160",0,120,120,1200,840.0,3.0,3.0,14,120,120,1380,1092.0,3.0,3.0
3,"2008-04-15 00:02:20,165",0,120,120,1380,1092.0,3.0,3.0,14,120,120,1140,1272.0,2.0,2.0
4,"2008-04-15 00:03:00,162",0,120,120,1140,1272.0,2.0,2.0,14,120,120,1080,1164.0,2.0,2.0
5,"2008-04-15 00:03:40,158",0,120,120,1080,1164.0,2.0,2.0,14,120,120,1140,1128.0,2.0,2.0
6,"2008-04-15 00:04:20,161",0,120,120,1140,1128.0,2.0,2.0,14,120,120,840,1056.0,2.0,2.0
7,"2008-04-15 00:05:00,165",0,120,120,840,1056.0,2.0,2.0,14,120,120,900,936.0,2.0,2.0
8,"2008-04-15 00:05:40,170",0,120,120,900,936.0,2.0,2.0,14,120,120,900,864.0,1.0,1.0
9,"2008-04-15 00:06:20,165",0,120,120,900,864.0,1.0,1.0,14,120,120,1020,972.0,2.0,2.0
